# Maru:

In [1]:
# !pip install maru

In [2]:
import maru

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


In [3]:
analyzer = maru.get_analyzer(tagger='rnn', lemmatizer='pymorphy')

In [4]:
analyzed = analyzer.analyze(["стали"])

In [5]:
for morph in analyzed:
    print(morph)

Morph(word='стали', lemma='стать', tag=Tag(pos=VERB,aspect=Perf,mood=Ind,number=Plur,tense=Past,verbform=Fin,voice=Act))


# IsaNLP:

In [6]:
# !pip install grpcio
# !pip install pyicu
# !pip3 install pycld2
# !pip install morfessor
# !polyglot download embeddings2.ru
# !polyglot download ner2.ru
# !pip install git+https://github.com/IINemo/isanlp.git
# !pip install git+https://github.com/IINemo/isanlp_srl_framebank

### Run this dockers:

In [7]:
# docker run --rm -p 3333:3333 inemo/isanlp
# docker run --rm --shm-size=1024m -ti -p 3334:9999 inemo/syntaxnet_rus server 0.0.0.0 9999
# docker run --rm -p 3335:3333 inemo/isanlp_srl_framebank

### Simple example:

In [8]:
from isanlp.processor_remote import ProcessorRemote
ppl = ProcessorRemote(host='localhost', port=3333, pipeline_name='default')

text_ru = 'Мама мыла раму'
annotations = ppl(text_ru)
print(annotations)

{'text': 'Мама мыла раму', 'lang': 'ru', 'tokens': [<isanlp.annotation.Token object at 0x7fc246e6a198>, <isanlp.annotation.Token object at 0x7fc246e6a1d0>, <isanlp.annotation.Token object at 0x7fc246e6a208>], 'sentences': [<isanlp.annotation.Sentence object at 0x7fc246e6a240>], 'postag': [['S,жен,од=им,ед', 'V,несов,пе=прош,ед,изъяв,жен', 'S,жен,неод=вин,ед']], 'lemma': [['мама', 'мыть', 'рама']]}


### Complex example:

In [9]:
from isanlp.pipeline_common import PipelineCommon
from isanlp.ru.processor_tokenizer_ru import ProcessorTokenizerRu
from isanlp.processor_sentence_splitter import ProcessorSentenceSplitter
from isanlp.ru.processor_mystem import ProcessorMystem
from isanlp.processor_polyglot import ProcessorPolyglot
from isanlp.processor_syntaxnet_remote import ProcessorSyntaxNetRemote

ppl = PipelineCommon([(ProcessorPolyglot().detect_language,
                       ['text'],
                       {0: 'lang'}),
                      (ProcessorTokenizerRu(), 
                       ['text'], 
                       {0 : 'tokens'}),
                      (ProcessorSentenceSplitter(), 
                       ['tokens'], 
                       {0 : 'sentences'}),
                      (ProcessorMystem(), 
                       ['tokens', 'sentences'], 
                       {'lemma' : 'lemma', 
                        'postag' : 'postag'}),
                      (ProcessorSyntaxNetRemote('localhost', 3334), 
                       ['tokens', 'sentences'],
                       {'syntax_dep_tree' : 'syntax_dep_tree'}),
                      (ProcessorPolyglot(), 
                       ['text'],
                       {'entities' : 'entities'})])

text_ru = 'Мама мыла раму.'
annotations = ppl(text_ru)

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


In [162]:
# annotations

### Simple example with SRL:

In [163]:
from isanlp_srl_framebank.pipeline_default import PipelineDefault 

In [673]:
sent = """Американский психолог А. Маслоу предложил смотреть на структуру человеческих потребностей как на пирамиду, в основании <…> лежат биологические потребности, а на вершине находится потребность в самореализации.. """

In [674]:
from joblib import Parallel, delayed

In [675]:
%%time

ppl = PipelineDefault(address_morph=('localhost', 3333),
                      address_syntax=('localhost', 3334),
                      address_srl=('localhost', 3335))

CPU times: user 250 µs, sys: 80 µs, total: 330 µs
Wall time: 340 µs


In [676]:
%%time

annotations = ppl(sent)

CPU times: user 17.6 ms, sys: 0 ns, total: 17.6 ms
Wall time: 1.72 s


In [677]:
%%time

# a = [ppl(sent) for i in range(10)]

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 17.2 µs


In [678]:
%%time

# b = Parallel(n_jobs=4)(delayed(ppl)(sent) for i in range(100))

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.58 µs


In [679]:
import os
import json

from convert_srl_to_brat import SrlConverter

In [680]:
brat_json = SrlConverter()(annotations)

In [681]:
# brat_json

In [682]:
# annotations

## Visuazization of BRAT:

In [683]:
# !git clone https://github.com/chorusai/brave.git
# !cd brave && python setup.py install

In [684]:
from brave import brave

In [685]:
from brave import brave, start_notebook_mode, brave_compare
start_notebook_mode()

In [686]:
coll_data = brat_json['coll_data']
doc_data = brat_json['doc_data']

In [687]:
brave(doc_data, coll_data)

In [688]:
ws = annotations['syntax_dep_tree'][0][0]

In [636]:
ws.parent

6

In [637]:
ws.link_name

'advmod'